# Mount drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# !ls /content/gdrive/My\ Drive

In [ ]:
# !unzip /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319/hw1_data.zip -d /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319

# Import Library

In [5]:
import os

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import json

from tqdm import tqdm
from datetime import datetime

In [6]:
import torch_geometric
from torch_geometric.data import Data, DataLoader
import torch_geometric.utils as utils

In [3]:
import networkit as nk

In [4]:
import networkx as nx

# Setting

In [7]:
class Setting:
    _root = os.getcwd()

    _data = os.path.join(_root, "hw1_data")

    data_synthetic = os.path.join(_data, "Synthetic", "5000")
    data_youtube = os.path.join(_data, "youtube")
    
    
     # Create dir for train/test
    date_time = datetime.strftime(datetime.now(), "%Y-%m-%d %H-%M")
    root = os.path.join(_root, "result", date_time)
    if os.path.exists(root):
        pass
    else:
        os.makedirs(root)


    ### Save plt info
    train_info_p = os.path.join(root, "train.json")
    val_info_p = os.path.join(root, "valid.json")
    test_info_p = os.path.join(root, "test.json")

    ### Save plt img
    result_plt_p = os.path.join(root, "train_plt.png")
    test_plt_p = os.path.join(root, "test_plt.png")
    sum_box_p = os.path.join(root, "sum_box.png")
        
        
    ### Data split
    data_split = os.path.join(_root, "split.json")
    

    # Setting of training
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    mini_epochs = 500
    epochs = 10000

    criterion = nn.BCELoss()
    weight_node2vec = os.path.join(root, "node2vec.pth")
    weight_drbc = os.path.join(root, "drbc.pth")
    
    params_drbc = None
    params_node2vec = None

    

In [8]:
setting = Setting()

# Dataset

## Data

- data.x	节点特征，维度是[num_nodes, num_node_features]。
- data.edge_index	维度是[2, num_edges]，描述图中节点的关联关系，每一列对应的两个元素，分别是边的起点和重点。数据类型是torch.long。需要注意的是，data.edge_index是定义边的节点的张量（tensor），而不是节点的列表（list）。
- data.edge_attr	边的特征矩阵，维度是[num_edges, num_edge_features]
- data.y	训练目标（维度可以是任意的）。对于节点相关的任务，维度为[num_nodes, *]；对于图相关的任务，维度为[1,*]。
- data.position	节点位置矩阵（Node position matrix），维度为[num_nodes, num_dimensions]。

- [Learning to Identify High Betweenness Centrality Nodes from
Scratch: A Novel Graph Neural Network Approach](https://arxiv.org/pdf/1905.10418.pdf)
- node initial feature = [$(d_v), 1, 1]

### Load all data

In [127]:
def load_data(path):

    edge_index = []
    between = []
    for f in os.listdir(path):
        if "score" in f:
            # ground truth of betweenness centrality
            p = os.path.join(path, f)
            between.append(p)
            pass
        else:
            p = os.path.join(path, f)
            edge_index.append(p)

    between.sort()
    edge_index.sort()
    
    return edge_index, between
    
    
synthetic, between = load_data(setting.data_synthetic)
yt, yt_score = load_data(setting.data_youtube)

### Split Data

In [128]:
from sklearn.model_selection import train_test_split


def split_data(path, x, y, replace=False):
    if os.path.exists(path) and replace:
        pass
    else:
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15)

        split = {
            "X_train": X_train,
            "X_valid": X_valid,
            "X_test": X_test,
            "y_train": y_train,
            "y_valid": y_valid,
            "y_test": y_test
        }

        with open(path, 'w') as f:
            json.dump(split, f)
            
split_data(path=setting.data_split, x=synthetic, y=between)

### Cvt 2 dataloader

In [129]:
def to_dataloader(x, y, batch):
    
    data_list = []
    for index, f in enumerate(x):
        edge_index = torch_geometric.io.read_txt_array(f, dtype=torch.long)
        edge_index = edge_index.t().contiguous()
        edge_index = utils.to_undirected(edge_index)

        row, col = edge_index  
        deg = utils.degree(col) # must use col to get degree, why?
        deg = deg.numpy()  

        vertice = []
        for d in deg:
            vertice.append([d, 1, 1])
        vertice = np.array(vertice, dtype=np.float)
        vertice = torch.from_numpy(vertice)

        ### between centrality
        bcs = []
        bc = torch_geometric.io.read_txt_array(y[index], dtype=torch.double)
        bc = bc.t().contiguous()
        row, col = bc
        bc = col
        bc = bc.numpy()
        
        bcs = torch.from_numpy(
                    np.array([[b] for b in bc], dtype=np.float))

        data = Data(x=vertice, edge_index=edge_index, y=bcs)
        data_list.append(data)

    loader = DataLoader(data_list, batch_size=batch)
    return data_list

### Cvt 2 Data

In [130]:
def to_data(x, y):
    edge_index = torch_geometric.io.read_txt_array(x, dtype=torch.long)
    edge_index = edge_index.t().contiguous()
    edge_index = utils.to_undirected(edge_index)

    row, col = edge_index  
    deg = utils.degree(col) # must use col to get degree, why?
    deg = deg.numpy()  

    vertice = []
    for d in deg:
        vertice.append([d, 1, 1])
    vertice = np.array(vertice, dtype=np.float)
    vertice = torch.from_numpy(vertice)

    ### between centrality
    bcs = []
    bc = torch_geometric.io.read_txt_array(y, dtype=torch.double)
    bc = bc.t().contiguous()
    row, col = bc
    bc = col
    bc = bc.numpy()
    bcs = torch.from_numpy(
            np.array([[b] for b in bc], dtype=np.float))

    data = Data(x=vertice, edge_index=edge_index, y=bcs)
    
    return data

#### Read data

In [131]:
f = open(setting.data_split, 'r')
data_split = json.load(f)

X_train = data_split["X_train"]
X_valid = data_split["X_valid"]
X_test = data_split["X_test"]
y_train = data_split["y_train"]
y_valid = data_split["y_valid"]
y_test = data_split["y_test"]

f.close()

#### Cvt

In [132]:
train_loader = to_dataloader(X_train, y_train, batch=1)
valid_loader = to_dataloader(X_valid, y_valid, batch=1)
test_loader = to_dataloader(X_test, y_test, batch=1)

#### Use all data

In [133]:
all_loader = to_dataloader(synthetic, between, batch=1)

#### Use YT data

In [134]:
# yt_loader = to_dataloader(yt, yt_score, batch=1)

# Model

## Hyper params

In [135]:
setting.params_drbc = dict(
    # according to source paper
    encoder_params = dict(
        c = 3,
        p = 128,
        num_layers = 5,
        device = setting.device
    ),
    decoder_params = dict(
        p = 128,
        q = 64
    )
)

# setting.params_node2vec = dict(
#     edge_index, 
#     embedding_dim, 
#     walk_length = 10, 
#     context_size,
#     p = 1,
#     q = 2
# )

## DrBC

In [136]:
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.typing import Adj, OptTensor

In [137]:
class Encoder(MessagePassing):
    def __init__(self, c, p, num_layers, device, aggr="add"):
        super(Encoder, self).__init__(aggr=aggr)
        
        self.num_layers = num_layers
        self.w_0 = nn.Linear(in_features=c, out_features=p).double()
        
        self.relu = nn.ReLU(inplace=True)
        self.rnn = nn.GRUCell(p, p).double()

        self.device = device

    def forward(self, data):
        
        x, edge_index = data.x, data.edge_index
        
        
        # compute dgree
        row, col = edge_index
        deg = utils.degree(col)
        deg = torch.add(deg, 1)
        deg_inv_sqrt = torch.pow(deg, -0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        
        # h_0 = x

        # h_1
        x = self.w_0(x)
        x = self.relu(x)
        x = F.normalize(x, p=2, dim=1)
        
        h_s = [x]
        
        for i in range(self.num_layers-1):
            # internally calls the message(), aggregate() and update() functions
            x = self.propagate(edge_index, x=x, norm=norm)
            h_s.append(x)
        
        h_s = torch.stack(h_s, dim=-1)
        # Use torch.max to replace max_pooling
        z, _ = torch.max(h_s, dim=-1)
        # z = global_max_pool(h_s, torch.tensor([0], dtype=torch.long).to(self.device))
        
        return z

    def message(self, x_j, norm: OptTensor):
        """      
        In addition, tensors passed to propagate() can be mapped to the respective nodes i and j 
        by appending _i or _j to the variable name, .e.g. x_i and x_j. 
        Note that we generally refer to i as the central nodes that aggregates information, 
        and refer to j as the neighboring nodes, since this is the most common notation.
        """
  
        return x_j if norm is None else norm.view(-1, 1) * x_j
    
    
    def update(self, aggr_out, x):
        """        
        Takes in the output of aggregation as first argument 
        and any argument which was initially passed to propagate().
        """

        x = self.rnn(x, aggr_out)
        x = F.normalize(x, p=2, dim=1) 
        
        return x
    

In [138]:
class Decoder(nn.Module):
    def __init__(self, p, q):
        
        super().__init__()
        
        self.w_4 = nn.Linear(in_features=p, out_features=q).double()
        self.w_5 = nn.Linear(in_features=q, out_features=1).double()
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, z):
        z = self.w_4(z)
        z = self.relu(z)
        z = self.w_5(z)
        
        return z

In [139]:
class DrBC(nn.Module):
    def __init__(self, encoder_params, decoder_params):
        super().__init__()
        
        self.encoder = Encoder(**encoder_params)
        self.decoder = Decoder(**decoder_params)
        
        
    def forward(self, data):
        
        z = self.encoder(data)
        
        return self.decoder(z)

## Node2Vec

In [140]:
from torch_geometric.nn import Node2Vec

In [158]:
class BC_node2vec(nn.Module):
    
    def __init__(self, node2vec_params):
        super().__init__()
        
        self.node2vec = Node2Vec(**node2vec_params)
        self.mlp = nn.Linear(in_features=128, out_features=1)
        
    def forward(self, data):
        
        x = self.node2vec(data)
        x = self.mlp(x)
        
        return x

# Load model

In [159]:
def load_checkpoint(filepath, device, **params):

    if "node2vec" in filepath:
        model = BC_node2vec(**params["node2vec"])
    
    else:
        model = DrBC(**params["drbc"])
    
    model = model.to(setting.device)
    
    if os.path.exists(filepath):
        print("pretrained finded")
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint['model_stat'])
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        optimizer.load_state_dict(checkpoint['optimizer_stat'])

    else:
        print("use a new optimizer")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    return model, optimizer

# Evaluation

### Top k %

In [160]:
def top_n(model, data, k):

    with torch.no_grad():

        data = data.to(setting.device)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))
        
        nodes = bc_gt.size()[0]
        k = int(nodes * k / 100)

        bc_pr = model(data)

        bc_gt = bc_gt.squeeze().to(setting.device)
        bc_pr = bc_pr.squeeze()

        gt_value, gt_indice = torch.topk(bc_gt, k)
        pr_value, pr_indice = torch.topk(bc_pr, k)

        gt_indice = set(gt_indice.cpu().numpy())
        pr_indice = set(pr_indice.cpu().numpy())

        intersect = len(gt_indice & pr_indice)
        top = intersect/k
            
       

        return top

### Tau Kendal

In [161]:
from scipy import stats

def tau_distance(model, data):
    
    
    with torch.no_grad():
 
        data = data.to(setting.device)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))

        bc_pr = model(data)

        bc_gt = bc_gt.squeeze().cpu().numpy()
        bc_pr = bc_pr.squeeze().cpu().numpy()


        tau, p_value = stats.kendalltau(bc_gt, bc_pr)
       

    return tau

### Evaluation Mode

In [162]:
def eval_model(model, data):
    
    with torch.no_grad():
        data = data.to(setting.device)
        bc_pr = model(data)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))
        
        nodes = bc_gt.size()[0]

        # random sample 5|V| nodes
        src = (torch.rand(nodes * 5) * (nodes-1)).long()
        det = (torch.rand(nodes * 5) * (nodes-1)).long()

        y_gt = (bc_gt[det] - bc_gt[src]).squeeze().to(setting.device)
        y_pr = (bc_pr[det] - bc_pr[src]).squeeze()

        y_gt = nn.Sigmoid()(y_gt)
        y_pr = nn.Sigmoid()(y_pr)

        loss = nn.BCELoss(reduction="sum")(y_pr, y_gt)
        
        return loss.item()

In [163]:
def from_networkx(G, between_list):
    r"""Converts a :obj:`networkx.Graph` or :obj:`networkx.DiGraph` to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        G (networkx.Graph or networkx.DiGraph): A networkx graph.
    """

    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G
    edge_index = torch.LongTensor(list(G.edges)).t().contiguous()
   
    data = {}

    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        for key, value in feat_dict.items():
           
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        for key, value in feat_dict.items():
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]

    for key, item in data.items():
        try:
            data[key] = torch.tensor(item)
        except ValueError:
            pass

    data['edge_index'] = edge_index.view(2, -1)
    data['x'] = torch.from_numpy(
        np.array( [ [G.degree[i], 1, 1] for i in G.nodes()], dtype=np.float ) )
    data['y'] = torch.from_numpy(
        np.array( [ [b] for b in between_list ] , dtype=np.float) )
    data = torch_geometric.data.Data.from_dict(data)
    data.num_nodes = G.number_of_nodes()

    return data

# Train

In [164]:
model, optimizer = load_checkpoint(
                    setting.weight_drbc,
                    setting.device,
                    drbc = setting.params_drbc,
                    node2vec = setting.params_node2vec)

use a new optimizer


In [ ]:
nodes_cnt = 5000


model.train()

train_info = {
    "bce": [],
    "top1": [],
    "top5": [],
    "top10": [],
    "kendal": []
}

valid_info = {
    "bce": [],
    "top1": [],
    "top5": [],
    "top10": [],
    "kendal": []
}

top_10 = 0.0
top_5 = 0.0
top_1 = 0.0

for epoch in range(setting.epochs):
    
    # Draw network G from distribution D (like the power-law model)
    G = nx.generators.random_graphs.powerlaw_cluster_graph(n=nodes_cnt, m=4, p=0.05)
    # Calculate each node’s exact BC value bv, ∀v ∈ V
    betweenness = nx.algorithms.centrality.betweenness_centrality(G)
    
    # Convert betweenness dict to list
    between_list = [v for k, v in sorted(betweenness.items(), key=lambda  item: int(item[0]), reverse=False)]
    
    # Convert networkx.Graph to Pyg Data
    pyg_data = from_networkx(G, between_list)
    print("\nEpoch: {}".format(epoch+1))
    
    

    
    ### Start training
    optimizer.zero_grad()

    pyg_data = pyg_data.to(setting.device)
    bc_pr = model(pyg_data)

    bc_gt = pyg_data.y
#     bc_gt = np.array(bc_gt)
#     bc_gt = torch.from_numpy(bc_gt)
    bc_gt = torch.reshape(bc_gt, (-1, ))

    # random sample 5|V| nodes
    src = (torch.rand(nodes_cnt * 5) * (nodes_cnt-1)).long()
    det = (torch.rand(nodes_cnt * 5) * (nodes_cnt-1)).long()

    y_gt = (bc_gt[det] - bc_gt[src]).squeeze().to(setting.device)
    y_pr = (bc_pr[det] - bc_pr[src]).squeeze()

    y_gt = nn.Sigmoid()(y_gt)
    y_pr = nn.Sigmoid()(y_pr)

    loss = nn.BCELoss(reduction="sum")(y_pr, y_gt)

    loss.backward()
    optimizer.step()
    
    
    ### Evaluation:- Train
    top1 = top_n(model, pyg_data, k=1)
    top5 = top_n(model, pyg_data, k=5)
    top10 = top_n(model, pyg_data, k=10)
    kendal = tau_distance(model, pyg_data)

    print("Train: loss = {}, top 1 = {}, top 5 = {}, top 10 = {}, kendal = {}".format(
            loss.item(),
            top1,
            top5,
            top10,
            kendal
    ))

    train_info["bce"].append(loss.item())
    train_info["top1"].append(top1)
    train_info["top5"].append(top5)
    train_info["top10"].append(top10)
    train_info["kendal"].append(kendal)

    with open(setting.train_info_p, 'w') as f:
        json.dump(train_info, f)



    ### Evaluation:- Synthetic
    bce_l = []
    top1_l = []
    top5_l = []
    top10_l = []
    kendal_l = []

    for data in all_loader:
        bce_l.append(eval_model(model, data))
        top1_l.append(top_n(model, data, k=1))
        top5_l.append(top_n(model, data, k=5))
        top10_l.append(top_n(model, data, k=10))
        kendal_l.append(tau_distance(model, data))
        
    bce = sum(bce_l)/len(bce_l)
    top1 = sum(top1_l)/len(top1_l)
    top5 = sum(top5_l)/len(top5_l)
    top10 = sum(top10_l)/len(top10_l)
    kendal = sum(kendal_l)/len(kendal_l)
        
    print("Valid: loss = {}, top 1 = {}, top 5 = {}, top 10 = {}, kendal = {}".format(
        bce,
        top1,
        top5,
        top10,
        kendal
    ))

    valid_info["bce"].append(bce)
    valid_info["top1"].append(top1)
    valid_info["top5"].append(top5)
    valid_info["top10"].append(top10)
    valid_info["kendal"].append(kendal)

    with open(setting.val_info_p, 'w') as f:
        json.dump(valid_info, f)
        
    
    if top10 > top_10:
        checkpoint = {
            'model_stat': model.state_dict(),
            'optimizer_stat': optimizer.state_dict(),
        }
        torch.save(checkpoint, setting.weight_drbc)
        top_10 = top10


Epoch: 1
Train: loss = 17328.681817183424, top 1 = 0.92, top 5 = 0.84, top 10 = 0.746, kendal = 0.7468194438887777
Valid: loss = 17328.69308357833, top 1 = 0.636, top 5 = 0.6187999999999999, top 10 = 0.6415333333333334, kendal = 0.4567827058745083

Epoch: 2
Train: loss = 17328.672852006828, top 1 = 0.86, top 5 = 0.76, top 10 = 0.584, kendal = 0.6932236047209441
Valid: loss = 17328.678468844235, top 1 = 0.632, top 5 = 0.6170666666666668, top 10 = 0.6380666666666667, kendal = 0.471629451223578

Epoch: 3
Train: loss = 17328.668453861283, top 1 = 0.86, top 5 = 0.728, top 10 = 0.538, kendal = 0.5157101820364073
Valid: loss = 17328.672009743066, top 1 = 0.6326666666666667, top 5 = 0.6165333333333335, top 10 = 0.6337333333333334, kendal = 0.47577902247116083

Epoch: 4
Train: loss = 17328.66575137698, top 1 = 0.9, top 5 = 0.7, top 10 = 0.522, kendal = 0.42890930186037207
Valid: loss = 17328.668630279673, top 1 = 0.6433333333333334, top 5 = 0.6184, top 10 = 0.6296666666666668, kendal = 0.47144


Epoch: 33
Train: loss = 17328.653448325822, top 1 = 0.9, top 5 = 0.848, top 10 = 0.844, kendal = 0.4620072814562912
Valid: loss = 17328.65568434698, top 1 = 0.6533333333333332, top 5 = 0.6134666666666667, top 10 = 0.6135333333333334, kendal = 0.10557763819430553

Epoch: 34
Train: loss = 17328.64469422565, top 1 = 0.92, top 5 = 0.828, top 10 = 0.834, kendal = 0.42570546109221846
Valid: loss = 17328.65531415783, top 1 = 0.6533333333333332, top 5 = 0.6128, top 10 = 0.599, kendal = 0.06491908515036343

Epoch: 35
Train: loss = 17328.646643883847, top 1 = 0.9, top 5 = 0.832, top 10 = 0.842, kendal = 0.3996640928185637
Valid: loss = 17328.654454256175, top 1 = 0.6526666666666666, top 5 = 0.6123999999999999, top 10 = 0.5848666666666668, kendal = 0.030116770020670803

Epoch: 36
Train: loss = 17328.636434249485, top 1 = 0.9, top 5 = 0.812, top 10 = 0.824, kendal = 0.37483928785757153
Valid: loss = 17328.65516279924, top 1 = 0.6526666666666666, top 5 = 0.6120000000000001, top 10 = 0.575866666666

Valid: loss = 17328.650445909854, top 1 = 0.6526666666666667, top 5 = 0.6192000000000003, top 10 = 0.5700666666666667, kendal = -0.0009162045742481831

Epoch: 65
Train: loss = 17328.635026567772, top 1 = 0.92, top 5 = 0.852, top 10 = 0.808, kendal = 0.5988834566913384
Valid: loss = 17328.65112122398, top 1 = 0.6526666666666667, top 5 = 0.6193333333333335, top 10 = 0.5793999999999999, kendal = 0.013391312929252518

Epoch: 66
Train: loss = 17328.637849889554, top 1 = 0.94, top 5 = 0.848, top 10 = 0.84, kendal = 0.6226557311462292
Valid: loss = 17328.64951921225, top 1 = 0.6526666666666667, top 5 = 0.6193333333333333, top 10 = 0.5851333333333334, kendal = 0.025466917383476693

Epoch: 67
Train: loss = 17328.647424285806, top 1 = 0.86, top 5 = 0.864, top 10 = 0.848, kendal = 0.6228559311862373
Valid: loss = 17328.650063206605, top 1 = 0.6533333333333334, top 5 = 0.6197333333333334, top 10 = 0.5826666666666667, kendal = 0.022930767486830705

Epoch: 68
Train: loss = 17328.633303855582, top 1 


Epoch: 96
Train: loss = 17328.61727549921, top 1 = 0.88, top 5 = 0.828, top 10 = 0.83, kendal = 0.5713630726145229
Valid: loss = 17328.647993295075, top 1 = 0.6533333333333334, top 5 = 0.6198666666666668, top 10 = 0.5177333333333334, kendal = -0.08645762219110488

Epoch: 97
Train: loss = 17328.632443088798, top 1 = 0.84, top 5 = 0.824, top 10 = 0.816, kendal = 0.5506758951790358
Valid: loss = 17328.649827784575, top 1 = 0.654, top 5 = 0.6192, top 10 = 0.5006666666666667, kendal = -0.1103756324598253

Epoch: 98
Train: loss = 17328.62134168735, top 1 = 0.88, top 5 = 0.868, top 10 = 0.776, kendal = 0.4926410882176436
Valid: loss = 17328.64796650268, top 1 = 0.654, top 5 = 0.6180000000000001, top 10 = 0.48406666666666676, kendal = -0.13334729345869173

Epoch: 99
Train: loss = 17328.62901662892, top 1 = 0.92, top 5 = 0.864, top 10 = 0.78, kendal = 0.46630126025205043
Valid: loss = 17328.6479226341, top 1 = 0.654, top 5 = 0.6169333333333333, top 10 = 0.4703333333333334, kendal = -0.14880897


Epoch: 128
Train: loss = 17328.627635227444, top 1 = 0.88, top 5 = 0.848, top 10 = 0.86, kendal = 0.5797671534306862
Valid: loss = 17328.645724094138, top 1 = 0.6533333333333334, top 5 = 0.6154666666666667, top 10 = 0.4597333333333334, kendal = -0.16499992265119698

Epoch: 129
Train: loss = 17328.61687286776, top 1 = 0.88, top 5 = 0.872, top 10 = 0.832, kendal = 0.5867437487497499
Valid: loss = 17328.647467225128, top 1 = 0.6533333333333334, top 5 = 0.6177333333333332, top 10 = 0.4717333333333333, kendal = -0.15251286523971463

Epoch: 130
Train: loss = 17328.624414062575, top 1 = 0.9, top 5 = 0.88, top 10 = 0.846, kendal = 0.6012298459691939
Valid: loss = 17328.64583878086, top 1 = 0.6546666666666668, top 5 = 0.6178666666666666, top 10 = 0.4761333333333334, kendal = -0.1520206441288258

Epoch: 131
Train: loss = 17328.6287462553, top 1 = 0.88, top 5 = 0.876, top 10 = 0.846, kendal = 0.5669354670934187
Valid: loss = 17328.64578153217, top 1 = 0.6553333333333334, top 5 = 0.61306666666666

...

Valid: loss = 17328.651911966605, top 1 = 0.6533333333333334, top 5 = 0.44733333333333325, top 10 = 0.26666666666666666, kendal = -0.2761543935453758

Epoch: 223
Train: loss = 17328.611765380432, top 1 = 0.88, top 5 = 0.856, top 10 = 0.866, kendal = 0.6281603520704141
Valid: loss = 17328.651604496506, top 1 = 0.6546666666666667, top 5 = 0.4996, top 10 = 0.30393333333333333, kendal = -0.2635042528505701

Epoch: 224
Train: loss = 17328.62901912673, top 1 = 0.86, top 5 = 0.864, top 10 = 0.876, kendal = 0.6460124024804962
Valid: loss = 17328.649258097215, top 1 = 0.6553333333333333, top 5 = 0.5438666666666666, top 10 = 0.3406, kendal = -0.249837599519904

Epoch: 225
Train: loss = 17328.616904429087, top 1 = 0.92, top 5 = 0.884, top 10 = 0.866, kendal = 0.6403992798559712
Valid: loss = 17328.648426299504, top 1 = 0.6553333333333333, top 5 = 0.5476000000000002, top 10 = 0.34579999999999994, kendal = -0.24537995599119827

Epoch: 226
Train: loss = 17328.622889044324, top 1 = 0.88, top 5 = 0.88

Valid: loss = 17328.65001853891, top 1 = 0.6613333333333333, top 5 = 0.5161333333333333, top 10 = 0.3173333333333332, kendal = -0.25039254250850174

Epoch: 255
Train: loss = 17328.625561192533, top 1 = 0.84, top 5 = 0.912, top 10 = 0.886, kendal = 0.6390913382676536
Valid: loss = 17328.64979638548, top 1 = 0.6606666666666666, top 5 = 0.5125333333333334, top 10 = 0.3139333333333333, kendal = -0.2500228152297126

Epoch: 256
Train: loss = 17328.631292813523, top 1 = 0.92, top 5 = 0.872, top 10 = 0.882, kendal = 0.6037889177835567
Valid: loss = 17328.64968594557, top 1 = 0.6606666666666666, top 5 = 0.4826666666666667, top 10 = 0.2918666666666666, kendal = -0.2600393625391745

Epoch: 257
Train: loss = 17328.632775026505, top 1 = 0.92, top 5 = 0.9, top 10 = 0.878, kendal = 0.5671404680936187
Valid: loss = 17328.652598423494, top 1 = 0.6606666666666666, top 5 = 0.4593333333333333, top 10 = 0.27499999999999997, kendal = -0.2679639661265586

Epoch: 258
Train: loss = 17328.618100438835, top 1 = 

Valid: loss = 17328.65222733973, top 1 = 0.6606666666666666, top 5 = 0.5170666666666667, top 10 = 0.31873333333333326, kendal = -0.24802724278188976

Epoch: 287
Train: loss = 17328.61794592531, top 1 = 0.98, top 5 = 0.888, top 10 = 0.87, kendal = 0.664119223844769
Valid: loss = 17328.65027802395, top 1 = 0.6606666666666666, top 5 = 0.4948000000000001, top 10 = 0.3009333333333333, kendal = -0.2558386077215443

Epoch: 288
Train: loss = 17328.622974163227, top 1 = 0.9, top 5 = 0.892, top 10 = 0.856, kendal = 0.603302100420084
Valid: loss = 17328.6508315399, top 1 = 0.662, top 5 = 0.44506666666666667, top 10 = 0.2670666666666666, kendal = -0.2722475321731013

Epoch: 289
Train: loss = 17328.628159126285, top 1 = 0.9, top 5 = 0.892, top 10 = 0.852, kendal = 0.5483731946389278
Valid: loss = 17328.651127799913, top 1 = 0.6613333333333333, top 5 = 0.42546666666666666, top 10 = 0.25320000000000004, kendal = -0.27868294192171766

Epoch: 290
Train: loss = 17328.619437271613, top 1 = 0.9, top 5 = 0


Epoch: 318
Train: loss = 17328.611805405035, top 1 = 0.96, top 5 = 0.88, top 10 = 0.862, kendal = 0.5775363072614523
Valid: loss = 17328.65253211084, top 1 = 0.6606666666666666, top 5 = 0.4553333333333335, top 10 = 0.2720666666666666, kendal = -0.26553802493832096

Epoch: 319
Train: loss = 17328.63150359154, top 1 = 0.88, top 5 = 0.888, top 10 = 0.88, kendal = 0.4078122824564913
Valid: loss = 17328.65395489619, top 1 = 0.6606666666666666, top 5 = 0.4233333333333333, top 10 = 0.25060000000000004, kendal = -0.2751738294325532

Epoch: 320
Train: loss = 17328.620417600207, top 1 = 0.96, top 5 = 0.888, top 10 = 0.892, kendal = 0.6069759551910382
Valid: loss = 17328.652191187182, top 1 = 0.6606666666666666, top 5 = 0.4593333333333333, top 10 = 0.2764666666666667, kendal = -0.26128201640328064

Epoch: 321
Train: loss = 17328.63479810419, top 1 = 0.92, top 5 = 0.884, top 10 = 0.882, kendal = 0.6651936787357472
Valid: loss = 17328.65340653601, top 1 = 0.6613333333333333, top 5 = 0.4968, top 10

Valid: loss = 17328.649393302836, top 1 = 0.6606666666666666, top 5 = 0.46226666666666677, top 10 = 0.2775333333333333, kendal = -0.2656012962592519

Epoch: 350
Train: loss = 17328.622462417952, top 1 = 0.96, top 5 = 0.864, top 10 = 0.878, kendal = 0.6800670534106822
Valid: loss = 17328.650743172493, top 1 = 0.6606666666666666, top 5 = 0.4689333333333333, top 10 = 0.28193333333333326, kendal = -0.2620128559045142

Epoch: 351
Train: loss = 17328.624884101206, top 1 = 0.96, top 5 = 0.908, top 10 = 0.886, kendal = 0.6787218243648729
Valid: loss = 17328.651544882057, top 1 = 0.6606666666666666, top 5 = 0.4710666666666667, top 10 = 0.28413333333333324, kendal = -0.2597194212175769

Epoch: 352
Train: loss = 17328.623727458362, top 1 = 0.94, top 5 = 0.916, top 10 = 0.886, kendal = 0.6854497299459892
Valid: loss = 17328.6509270537, top 1 = 0.6606666666666666, top 5 = 0.48960000000000004, top 10 = 0.29786666666666667, kendal = -0.2517260625458425

Epoch: 353
Train: loss = 17328.625439547934, to

# Visualization

In [ ]:
import matplotlib.pyplot as plt

with open(setting.train_info_p, 'r') as train_info, open(setting.valid_info_p, 'r') as valid_info:
    
    train = json.load(train_info)
    valid = json.load(valid_info)

    epochs = setting.epochs
    keys = list(train.keys())

    x = np.linspace(1, epochs, epochs)
    
    ### plot bce loss and mean top N %
    fig, axs = plt.subplots(len(keys), figsize=(25, 4 * len(keys)))
    for index, ax in enumerate(axs):
        key = keys[index]
        ax.plot(x, train[key], color="blue")
        ax.plot(x, valid[key], color="orange")
        ax.legend(["train", "valid"], loc='upper left')
        
#         if key == "bce":    
#         else:
#             y = [sum(top)/len(top) for top in train[key]]
#             ax.plot(x, y, color="blue")
        
        ax.set_title(key)
        ax.grid()
        
    plt.tight_layout()
    plt.savefig(setting.result_plt_p, transparent=True)

# Baseline

In [ ]:
import networkit as nk

In [ ]:
G = nk.readGraph("/hw1_data/Synthetic/5000/0.txt", nk.Format.EdgeListTabZero)

## RK (DIAM)

### ApproxBetweenness

Fast approximation of betweenness centrality through sampling.

In [ ]:
ab = nk.centrality.ApproxBetweenness(G, epsilon=0.1)
ab.run()

In [ ]:
# The 10 most central nodes according to betweenness are then 
ab.ranking()[:10]

## ABRA

Approximating betweenness centrality in static and dynamic graphs with rademacher averages

error tolerance λ to 0.01 and the probability δ to 0.1

## KADABRA

In [ ]:
# Initialize algorithm 
kadabra = nk.centrality.KadabraBetweenness(G, 0.05, 0.8) 
kadabra.run() 

In [ ]:
#The 10 most central nodes according to betweenness are then 
kadabra.ranking()[:10]

## K-BC (KPATH)

k-centralities: Local approximations of global measures based on shortest paths

In [ ]:
def KbcBetweenness(G):
    start = time.clock()
    kBC = dict({key:0 for key in G.nodes()})
    for s in G.nodes():
        #Structures
        Q = deque() #Queue
        S = deque() #Stack
        dist = dict({key:inf for key in G.nodes()})
        sig = dict({key:float(0) for key in G.nodes()})
        pred = dict({key:[] for key in G.nodes()})

        #Initialize
        Q.append(s)
        dist[s] = 0
        sig[s] = 1

        #BFS - top-down
        while len(Q) > 0:
            v = Q.popleft()
            if dist[v] > k:
                break;
            S.append(v)
            for w in G[v]:
                if dist[w] == inf:
                    dist[w] = dist[v] + 1
                    if dist[w] <= k:
                        Q.append(w)
                if dist[w] == dist[v] + 1:
                    sig[w] += sig[v]
                    pred[w].append(v)
        #Aggregate - bottom-up
        delta = dict({key:float(0) for key in G.nodes()})
        while len(S) > 0:
            v = S.pop()
            for w in pred[v]:
                delta[w] += (delta[v]+1) * sig[w] / sig[v]
            if v!=s:
                if(G.is_directed()):
                    kBC[v] += delta[v]
                else: kBC[v] += delta[v] / 2.0
    end = time.clock()
    print('My time: ' + str(end - start))